In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
from torch import Tensor
import torch.nn.functional as F

In [14]:
df_train = pd.read_csv("normalied_train_data.csv")

In [15]:
df_train.drop(df_train.columns[[0,1]],axis= 1,inplace = True)

In [16]:
df_train.head()

,hash,trajectory_id,time_entry,time_exit,x_entry,y_entry,x_exit,y_exit,label
0,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_0,-0.411531,-0.419661,-0.360248,-0.377866,2.485085,-0.377866,0
1,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_1,-0.400807,-0.408314,-0.496767,-0.481641,-0.360753,-0.481641,0
2,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_2,-0.377467,-0.350515,-0.482237,-0.484201,-0.287116,-0.484201,0
3,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_3,-0.326408,-0.321582,-0.482040,-0.484307,-0.283475,-0.484307,0
4,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_4,2.255426,2.196077,-0.481575,-0.482700,-0.263699,-0.482700,0


In [17]:
small = True
if small == True:
    df_train = df_train.head(10000)

# the following step tells the distribution of number of trajectories

In [18]:
# create all_trajs, which stores every trajectory in a list

# create a trajectory dictionary
# {key: "hash" value: ["x_entry","y_entry","x_exit","y_exit","time_entry","time_exit"] }

whole_dict = {}
all_trajs = []
for index, row in df_train.iterrows():
    
    all_trajs.append([row["x_entry"],row["y_entry"],row["x_exit"],row["y_exit"],row["time_entry"],row["time_exit"]])
   
    if row["hash"] not in whole_dict:
        whole_dict[row["hash"]] = []
    whole_dict[row["hash"]].append([row["x_entry"],row["y_entry"],row["x_exit"],row["y_exit"],row["time_entry"],row["time_exit"]])
        


In [19]:
from collections import Counter

lst = [len(value) for value in whole_dict.values()]
count = Counter(lst)
num_traj_distribution = count.most_common()

## the following code treat all the trajectory equally, i.e., there is no ID difference. the batch size = 64

In [20]:
# data preparation
X_train = all_trajs
y_train = df_train["label"].tolist()


In [21]:
# utility for getting prediction accuracy
def get_correct_and_accuracy(y_pred, y):
    # y_pred is the nxC prediction scores
    # give the number of correct and the accuracy
    n = y.shape[0]
    # find the prediction class label
    _ ,pred_class = y_pred.max(dim=1)
    correct = (pred_class == y).sum().item()
    return correct ,correct/n

In [22]:
class RNN(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers,batch_size):
        super(RNN, self).__init__()

        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.sequence_length = 1
        self.batch_size = batch_size

        self.rnn = nn.RNN(input_size=6, hidden_size=64, batch_first=True)
        self.fc = nn.Linear(self.hidden_size,2)
        self.hidden = self.init_hidden()
        
        
        
    def forward(self, x):

        # Reshape input
        x = x.reshape(self.batch_size, self.sequence_length, self.input_size)

        # Propagate input through RNN
        # Input: (batch, seq_len, input_size)
        x, self.hidden = self.rnn(x, self.hidden)
        x = self.fc(x)
        x = F.Softmax(x)
        x = torch.argmax(x, dim=1)
        
        return x

    def init_hidden(self):
        return(torch.zeros(self.num_layers, self.batch_size, self.hidden_size),   
                torch.zeros(self.num_layers, self.batch_size, self.hidden_size))

In [23]:
# Instantiate RNN model
num_classes = 2
input_size = 6
hidden_size=64
num_layers = 2
batch_size = 64

rnn = RNN(num_classes, input_size, hidden_size, num_layers, batch_size)


# Set loss and optimizer function
# CrossEntropyLoss = LogSoftmax + NLLLoss
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.005)

# X and Y preparation
X_train_tensor = Tensor(all_trajs)
Y_train_tensor = Tensor(y_train).long()

In [25]:
x = X_train_tensor[:64]
y_pred =rnn(x)
print(y_pred)

AttributeError: 'tuple' object has no attribute 'size'

# An alternative way, LSTM model

In [ ]:
class LSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1, num_layers=2, drop_prob = 0.6, lr= 0.005):
        super(LSTM, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        dropout = drop_prob
        
        # define LSTM
        self.lstm = nn.LSTM(
                            self.input_dim, 
                            self.hidden_dim, 
                            self.num_layers,
                            dropout = drop_prob
            #batch_first = True
                            )
        
        # define a dropout layper
        self.dropout = nn.Dropout(dropout)
        
        # Define the final fc output layer
        self.fc = nn.Linear(self.hidden_dim, output_dim)
        
        # initialize the weights
        # self.init_weights()
        

    def forward(self, x):
                
        # Reshape input (batch first)
        # x = x.view(self.batch_size, 1, self.input_dim)


        # get x and the hidden state from the lstm
        x = self.lstm(x.reshape(self.batch_size,-1,6))
        print("Type of x:",type(x))
        print("x:",x)
        # pass x through the dropout layer
        self.dropout(0.6)
        
        # reshape it for the last linear fully connected layer
        # x = x.view(-1,self.hidden_dim)
        # x = self.fc(x)
        x = self.fc(x.reshape(-1,output_dim))
        return x
        
    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

In [ ]:
# init network
lstm = LSTM(6,64,64)
print('model structure: ', lstm)
# init optimizer
optimizer = optim.Adam(lstm.parameters(),lr = 0.005)
# set loss function
criterion = nn.CrossEntropyLoss()


X_train_tensor = Tensor(X_train)
# X_val_tensor = T
y_train_tensor = Tensor(y_train).long() 
# y_val_tensor = Tensor(y_val).long()


# prepare for mini-batch stochastic gradient descent
n_iteration = 40
batch_size = 64
n_data = X_train_tensor.shape[0]
n_batch = int(np.ceil(n_data/batch_size))



print('X train tensor shape:', X_train_tensor.shape)

In [ ]:
## start 
train_loss_list = np.zeros(n_iteration)
train_accu_list = np.zeros(n_iteration)
val_loss_list = np.zeros(n_iteration)
val_accu_list = np.zeros(n_iteration)

hidden = lstm.init_hidden()
for i in range(n_iteration):
    # first get a minibatch of data
    
    total_train_loss = 0
    total_train_acc = 0
    
    for j in range(n_batch):
        batch_start_index = j*batch_size
        # get data batch from the normalized data
        X_batch = X_train_tensor[batch_start_index:batch_start_index+batch_size]
        print("x",X_batch.shape)
        
        # get ground truth label y
        y_batch = y_train_tensor[batch_start_index:batch_start_index+batch_size]
        
        y_pred = lstm.forward(X_batch)
        print("y:", y_pred)
        loss = criterion(y_pred,y_batch)
        total_train_loss += loss
        _ , accu = get_correct_and_accuracy(y_pred,y_batch)
        total_train_acc += accu
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # y_val_pred = conv_net.forward(X_val_tensor)
    ave_train_loss = total_train_loss/n_batch
    train_accu = total_train_acc/n_batch
    # val_loss = criterion(y_val_pred,y_val_tensor)
    # _,val_accu = get_correct_and_accuracy(y_val_pred,y_val_tensor)
        
        
    #print("Iter %d ,Train loss: %.3f, Train acc: %.3f, Val loss: %.3f, Val acc: %.3f"  %(i ,ave_train_loss, train_accu, val_loss, val_accu)) 
    ## add to the logs so that we can use them later for plotting
    train_loss_list[i] = ave_train_loss
    train_accu_list[i] = train_accu
    #val_loss_list[i] =  val_loss
    #val_accu_list[i] =  val_accu
    print("ave_train_loss:",ave_train_loss)
    print("train_accuracy:",train_accu)